In [7]:
import torch
print(torch.backends.mps.is_available())

True


In [8]:
from langchain_community.llms import Ollama
llm = Ollama(model="mistral")

llm.invoke("Tell me another joke")

" Why don't scientists trust atoms?\n\nBecause they make up everything!"

In [11]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer

# model_id = "mistralai/Mixtral-8x7B-v0.1"
# tokenizer = AutoTokenizer.from_pretrained(model_id)

# model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=True, device_map='mps')

# text = "Hello my name is"
# inputs = tokenizer(text, return_tensors="pt").to("mps")

# outputs = model.generate(**inputs, max_new_tokens=20)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [17]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [27]:
# base_model_id = "mistralai/Mistral-7B-v0.1"
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit = True,
#     bnb_4bit_use_double_quant = True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )
# model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map='mps')

In [25]:
torch.backends.mps.is_available()

True

In [22]:
client.close()

In [18]:
# from pathlib import Path
# import qdrant_client
# from llama_index import VectorStoreIndex, ServiceContext, SimpleDirectoryReader
# from llama_index.llms import Ollama
# from llama_index.storage.storage_context import StorageContext
# from llama_index.vector_stores.qdrant import QdrantVectorStore

# # Loading the documents from the disk
# documents = SimpleDirectoryReader("./data").load_data()

# # Initializing the vector store with Qdrant
# client = qdrant_client.QdrantClient(path="./qdrant_data")
# vector_store = QdrantVectorStore(client=client, collection_name="springboot")
# storage_context = StorageContext.from_defaults(vector_store=vector_store)

# # Initializing the Large Language Model (LLM) with Ollama
# # The request_timeout may need to be adjusted depending on the system's performance capabilities
# llm = Ollama(model="mistral", request_timeout=120.0)
# service_context = ServiceContext.from_defaults(llm=llm, embed_model="local")

# # Creating the index, which includes embedding the documents into the vector store
# index = VectorStoreIndex.from_documents(documents, service_context=service_context, storage_context=storage_context)

# # Querying the index with a specific question
# query_engine = index.as_query_engine()
# prompt = (
#   "What are the optimization methods, show me only the method name"
# )
# response = query_engine.query(prompt)
# client.close()
# print(response)

In [12]:
# Just runs .complete to make sure the LLM is listening
from llama_index.llms.ollama import Ollama

llm = Ollama(model="mistral")

response = llm.complete("What is the history of LEGO?")
print(response)

 LEGO is a Danish toy company that produces interlocking plastic bricks and other construction toys. The company was founded on August 10, 1932 by Ole Kirk Christiansen in Billund, Denmark. Initially, the company produced wooden toys, but during the late 1940s, they began manufacturing the first LEGO bricks using a plastic resin.

The name "LEGO" is derived from the Danish words "leg godt," which mean "play well." The interlocking bricks were first introduced in 1949 and were initially marketed under the name "Automatic Binding Bricks." However, the brand name LEGO did not officially appear until 1954.

In the 1960s, the company introduced various themes for its sets, including space exploration, castles, and towns. In 1968, they released their most famous set to date: the LEGO Modular House. This modular system allowed children to build houses with multiple rooms and stories by connecting pre-designed modules together.

Throughout the 1970s and 1980s, LEGO continued to expand its prod

In [19]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
)



In [20]:
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

In [21]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)
metadata_obj.create_all(engine)

In [22]:
from llama_index.core import SQLDatabase, ServiceContext
sql_database = SQLDatabase(engine, include_tables=["city_stats"])
from sqlalchemy import insert

rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {
        "city_name": "Chicago",
        "population": 2679000,
        "country": "United States",
    },
    {"city_name": "Seoul", "population": 9776000, "country": "South Korea"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [23]:
llm = Ollama(model="mistral")
service_context = ServiceContext.from_defaults(llm=llm,  embed_model="local")
sql_database = SQLDatabase(engine, include_tables=["city_stats"])

/var/folders/h3/n9s1wd956zs2c_l90d4zk39w0000gn/T/ipykernel_1281/2299164370.py:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm,  embed_model="local")
/Users/aswin/miniconda3/envs/mlenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████████████████████████| 684/684 [00:00<00:00, 245kB/s]
model.safetensors: 100%|█████████████████████| 133M/133M [00:16<00:00, 7.88MB/s]
tokenizer_config.json: 100%|████████████████████| 366/366 [00:00<00:00, 105kB/s]
vocab.txt: 100%|██████████████████████████████| 232k/232k [00:00<00:00, 586kB/s]
tokenizer.json: 100%|█████████████████████████| 711k/711k [0

In [24]:
stmt = select(
    city_stats_table.c.city_name,
    city_stats_table.c.population,
    city_stats_table.c.country,
).select_from(city_stats_table)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Chicago', 2679000, 'United States'), ('Seoul', 9776000, 'South Korea')]


In [25]:
from sqlalchemy import text

with engine.connect() as con:
    rows = con.execute(text("SELECT city_name from city_stats"))
    for row in rows:
        print(row)

('Chicago',)
('Seoul',)
('Tokyo',)
('Toronto',)


In [27]:
from llama_index.core.indices.struct_store.sql_query import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["city_stats"],
    service_context = service_context
)
query_str = "Which city has the highest population?"
response = query_engine.query(query_str)

In [28]:
response

Response(response=' Based on the input question and assuming we have a table named "city\\_stats" with columns "city\\_name" and "population," the correct SQL query would be as follows:\n\nSQL: SELECT city\\_name FROM city\\_stats ORDER BY population DESC LIMIT 1;\n\nResponse: The city with the highest population is [City Name] (assuming the first record returned from the query).', source_nodes=[NodeWithScore(node=TextNode(id_='87f50e35-ee10-4976-85cd-78ee3973dec0', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="Error: Statement 'SELECT city\\\\_name, population FROM city\\\\_stats ORDER BY population DESC LIMIT 1;' is invalid SQL.", start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'87f50e35-ee10-4976-85cd-78ee3973dec0': {}, 'sql_query': 'SELECT city\\_name, population FROM city\\_stats ORDER BY p

In [34]:
from llama_index.core.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)
from IPython.display import Markdown, display
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import VectorStoreIndex

# set Logging to DEBUG for more detailed outputs
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="city_stats"))
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
    service_context = service_context
)
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=5),
    service_context = service_context
)

response = query_engine.query("Which city has the highest population?")
display(Markdown(f"<b>{response}</b>"))

<b> I'm sorry for the confusion earlier. The given SQL query seems to be incorrect as there is no table named "city\_stats" specified in the query. Here is a corrected version of the SQL query assuming we have a table named "population\_data" with columns "city\_name" and "population":

SQL: SELECT city\_name FROM population\_data ORDER BY population DESC LIMIT 1;

Response: The city with the highest population is [City Name], according to the query results. (Replace "[City Name]" with the actual name of the city from the SQL response.)</b>

In [37]:
response.metadata['sql_query'].replace('\\', '')


'SELECT city_name, population FROM city_stats ORDER BY population DESC LIMIT 1;'

In [50]:
from llama_index.retrievers import NLSQLRetriever

# default retrieval (return_raw=True)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["city_stats"], return_raw=True,
    service_context = service_context  
)

results = nl_sql_retriever.retrieve(
    "Return the top 5 cities (along with their populations) with the highest population."
)
from llama_index.response.notebook_utils import display_source_node
for n in results:
    display_source_node(n, show_source_metadata=True)

**Node ID:** 17db383a-f008-4e1e-9f19-f7fda59a2c67<br>**Similarity:** None<br>**Text:** [('Tokyo', 13960000), ('Seoul', 9776000), ('Toronto', 2930000), ('Chicago', 2679000)]<br>**Metadata:** {}<br>

In [39]:
import sqlvalidator

sql_query = sqlvalidator.parse(response.metadata['sql_query'])

if not sql_query.is_valid():
    print(sql_query.errors)


In [52]:
from llama_index.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(nl_sql_retriever, service_context=service_context)
response = query_engine.query(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [53]:
print(str(response))

 To retrieve the top 5 cities with the highest population, you can use a SQL query in the following format:
```sql
SELECT city_name, population 
FROM city_stats 
ORDER BY population DESC 
LIMIT 5;
```
However, based on the context provided, it seems that there is an error in the syntax of the query. To correct this error, make sure to include a semicolon at the end of the statement:
```sql
SELECT city_name, population 
FROM city_stats 
ORDER BY population DESC 
LIMIT 5;
```
This corrected query will return the top 5 cities with the highest population from the `city_stats` table.
